In [1]:
# %pip install pypsa
# %pip install pandas
# %pip install dash

In [2]:
%reset -f
import pypsa
import warnings
import pandas as pd
import pypsabackground as pyp
import network_saver as nets
from pathlib import Path

# Set pandas options
pd.set_option('display.max_columns', None)
pd.options.plotting.backend = 'plotly'

# Configure warnings
warnings.filterwarnings("ignore", category=UserWarning, module='openpyxl')

In [3]:
'''     Set the Time Period for the Network
________________________________________________'''

timePeriod = {
    "startDate"     :   "2025-01-01 00:00",
    "yearAmount"    :   1
}

'''     Set the path for Input Data in Excel files for Folder Management
_____________________________________________________________________________'''

'''     -   Do not change folders unless necessary
        -   Move Input Data to their respective Folders before Running      '''

inputFolder = Path("InputData")  # Input Data is in this Folder
inputData = {
    "Plant": {
        "Folder"    :   "PlantData",       # Keep Folders the same for Convenience
        "File"      :   "AfricaModel_V4",                                            # Change File Names for new Data
        "Sheet"     :   "HydroAfrica"
    },
    "Demand": {
        "Folder"    :   "DemandData",
        "File"      :   "AfricaDemandData_V1"
    },
    "River": {
        "Folder"    :   "RiverFlow",
        "File"      :   "AfricaRiverData_V1"
    }
}

'''     Set the path for Data Exporting as CSV files:
___________________________________________________________'''

exportFolder = Path("ExportCSV")
exportData = {
    "Input" : {
        "Folder"    :   "InputCSV",
        "File"      :   "AfricaInputData"
    },
    "Result" : {
        "Folder"    :   "ResultCSV",
        "File"      :   "AfricaResultData"
    }
}

'''     Networks for the Webpage to Read are Saved in this Folder 
_______________________________________________________________________'''

saveNetFolder = Path("SavedNetworks") 

'''     Create Folders for Exporting
_________________________________________'''

saveNetFolder.mkdir(parents=True, exist_ok=True)
exportFolder.mkdir(parents=True, exist_ok=True)

In [4]:
'''     This Takes the Data Inputs Above and makes the Network
___________________________________________________________________'''
'''     Ignore [Error adding _____ for __: ___] 
        This Error is from Empty or Null Fields      '''

# Automatic Folder Pathing for Above Data Inputs
plantPath = nets.get_path(inputData, inputFolder, "Plant", "File")
riverPath = nets.get_path(inputData, inputFolder, "River", "File")
demandPath = nets.get_path(inputData, inputFolder, "Demand", "File")
start_date = pd.Timestamp(timePeriod["startDate"])
end_date = start_date + pd.DateOffset(years=timePeriod["yearAmount"]) - pd.Timedelta(hours=1)
snapshots = pd.date_range(start_date,end_date , freq="h")

# Selects a Solver
select_solver = pyp.solver_selected() # Default is normally 'highs'

# Create network
network = pypsa.Network()
network.set_snapshots(snapshots)

# Import Static, Plant, River and then Demand Data
pyp.import_from_excel(network, f"{str(plantPath)}.xlsx")
with pd.ExcelFile(f"{str(plantPath)}.xlsx") as xls:
    plant_data_df = pd.read_excel(xls, sheet_name=inputData["Plant"]["Sheet"], comment="#")
update_components = plant_data_df.apply(lambda row: pyp.add_plant_data(network, row), axis=1)
pyp.import_river_inflows(network, f"{str(riverPath)}.csv")
pyp.import_demand(network, f"{str(demandPath)}.csv")

Solver highs succeeded!
Selected solver: highs
Error adding link for 0: cannot convert float NaN to integer
Error adding link for 6: cannot convert float NaN to integer
Error adding link for 8: cannot convert float NaN to integer
Error adding link for 9: cannot convert float NaN to integer
Error adding link for 10: cannot convert float NaN to integer
Error adding link for 11: cannot convert float NaN to integer
Error adding link for 12: cannot convert float NaN to integer
Error adding link for 18: cannot convert float NaN to integer
Error adding link for 22: cannot convert float NaN to integer
Error adding link for 26: cannot convert float NaN to integer
Error adding link for 32: cannot convert float NaN to integer
Error adding link for 33: cannot convert float NaN to integer


In [5]:
'''     1 Export Input and Result Data as CSV
        2 Solves the Network
        3 Saves the Network as a .h5 file
___________________________________________________'''

nets.export_files(network, exportData, exportFolder, "Input", 'csv')
select_solver = pyp.solver_selected()
network.optimize(solver_name=select_solver)
nets.export_files(network, exportData, exportFolder, "Result", 'csv')
nets.export_files(network, '', saveNetFolder, "network", 'h5')

Solver highs succeeded!
Selected solver: highs


INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing binary variables.: 100%|██████████| 3/3 [00:00<00:00,  3.74it/s]
INFO:linopy.io: Writing time: 19.02s
INFO:linopy.solvers:Log file at C:\Users\dazzc\AppData\Local\Temp\highs.log
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 1322775 primals, 2119907 duals
Objective: 3.35e+10
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Generator-p_set, Link-com-p-lower, Link-com-p-upper, Link-com-transition-start-up, Link-com-transition-shut-down, Link-com-status-min_up_time_must_stay_up, Link-com-p0-ramp_limit_up, Link-com-p0-ramp_limit_down, Store-fix-e-lower, Store-fix-e-upper, Store-energy_balance were not assigned to the network.
INFO:pypsa.io:Exported network 'AfricaResultData2.csv' conta

Result saved to: ExportCSV\AfricaResultData2.csv


INFO:pypsa.io:Exported network 'network3.h5' contains: carriers, stores, loads, buses, generators, links


Network saved to: SavedNetworks\network3.h5
